In [3]:
import requests
import json

vllm_host = "http://localhost:8000"
url = f"{vllm_host}/v1/completions"

headers = {"Content-Type": "application/json"}
data = {
    "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "prompt": [
        "hi there!",
    ],
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json()) 

{'id': 'cmpl-c82abe22d03d46fd83ca138b7f25983d', 'object': 'text_completion', 'created': 1738555557, 'model': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', 'choices': [{'index': 0, 'text': '!!!!!!!!!!!!!!!!', 'logprobs': None, 'finish_reason': 'length', 'stop_reason': None, 'prompt_logprobs': None}], 'usage': {'prompt_tokens': 4, 'total_tokens': 20, 'completion_tokens': 16, 'prompt_tokens_details': None}}
